In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
from torchvision.transforms import Resize, Lambda, InterpolationMode
from torchvision.models import vgg13, VGG13_Weights

from sklearn.metrics import confusion_matrix, classification_report

from importlib.machinery import SourceFileLoader
pmodule = SourceFileLoader("*", "../preprocessing/preprocessing.py").load_module()

In [2]:
MODEL_NAME = "vgg13"

N_MELS = 128

BATCH_SIZE = 128
LEARNING_RATE = 1e-5
NUM_EPOCHS = 10

In [3]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [4]:
preprocess = pmodule.Preprocess("../dataset/IDMT_Traffic.zip")
X_train, X_val, X_test, y_train, y_val, y_test = preprocess.train_val_test_split()

train_dataset = pmodule.IDMTTrafficDataset(X_train, y_train, "../dataset/IDMT_Traffic.zip", n_mels=N_MELS, target_sample_rate=22050)
val_dataset = pmodule.IDMTTrafficDataset(X_val, y_val, "../dataset/IDMT_Traffic.zip", n_mels=N_MELS, target_sample_rate=22050)
test_dataset = pmodule.IDMTTrafficDataset(X_test, y_test, "../dataset/IDMT_Traffic.zip", n_mels=N_MELS, target_sample_rate=22050)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

Metadata for idmt_traffic_all.txt: 
Metadata for eusipco_2021_train.txt: 
Metadata for eusipco_2021_test.txt: 
X_trian: (4698, 6)	y_train: (4698,)
X_val: (1175, 6)		y_val: (1175,)
X_test: (2857, 6)	y_test: (2857,)


In [ ]:
model = vgg13(weights=VGG13_Weights.IMAGENET1K_V1)
num_ftrs = model.classifier[6].in_features
features = list(model.classifier.children())[:-1]
features.extend([nn.Linear(num_ftrs, 4)])
model.classifier = nn.Sequential(*features)
model.to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

def multi_acc(y_pred, y_true):
    y_pred_softmax = torch.log_softmax(y_pred, dim=1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim=1)    
    correct_pred = (y_pred_tags == y_true).float()
    acc = correct_pred.sum() / len(correct_pred)
    acc = torch.round(acc * 100)
    return acc

In [ ]:
train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in range(NUM_EPOCHS):
    train_running_loss = 0.0
    train_running_acc = 0.0
    val_running_loss = 0.0
    val_running_acc = 0.0

    # Train
    model.train()
    for idx, (train_features, y_train_trues) in enumerate(train_dataloader, 0):
        # Resize to 224x224
        X_train = Resize((224, 224), interpolation=InterpolationMode.BILINEAR)(train_features)

        X_train, y_train_trues = X_train.to(device), y_train_trues.to(device)

        # Foward pass
        optimizer.zero_grad()

        y_train_preds = model(X_train)

        train_loss = loss_fn(y_train_preds, y_train_trues)
        train_acc = multi_acc(y_train_preds, y_train_trues)

        train_loss.backward()
        optimizer.step()

        train_running_loss += train_loss.item()
        train_running_acc += train_acc.item()
    train_losses.append(train_running_loss/len(train_dataloader))
    train_accs.append(train_running_acc/len(train_dataloader))

    # Validation
    with torch.no_grad():
        model.eval()
        for idx, (val_features, y_val_trues) in enumerate(val_dataloader, 0):
            # Resize to 224x224
            X_val = Resize((224, 224), interpolation=InterpolationMode.BILINEAR)(val_features)

            X_val, y_val_trues = X_val.to(device), y_val_trues.to(device)

            # Forward pass
            y_val_preds = model(X_val)

            val_loss = loss_fn(y_val_preds, y_val_trues)
            val_acc = multi_acc(y_val_preds, y_val_trues)

            val_running_loss += val_loss.item()
            val_running_acc += val_acc.item()
    val_losses.append(val_running_loss/len(val_dataloader))
    val_accs.append(val_running_acc/len(val_dataloader))

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}]\t|\tTrain Loss: {train_running_loss/len(train_dataloader):.5f}\t|\tVal Loss: {val_running_loss/len(val_dataloader):.5f}\t|\tTrain Acc: {train_running_acc/len(train_dataloader):.3f}\t|\tVal Acc: {val_running_acc/len(val_dataloader):.3f}")


# Save the model
torch.save(model.state_dict(), f"idmt_{MODEL_NAME}_{N_MELS}mel_{LEARNING_RATE}lr_{NUM_EPOCHS}epoch_model.pt")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.plot(train_accs, '-o', label='Train')
ax1.plot(val_accs, '-o', label='Val')
ax1.set_title('Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

ax2.plot(train_losses, '-o', label='Train')
ax2.plot(val_losses, '-o', label='Val')
ax2.set_title('Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.savefig(f"idmt_{MODEL_NAME}_{N_MELS}mel_{LEARNING_RATE}lr_{NUM_EPOCHS}epoch_model.png")

In [ ]:
y_test_pred_list = []
with torch.no_grad():
    model.eval()
    for idx, (test_features, y_test_trues) in enumerate(test_dataloader, 0):
        # Resize to 224x224
        X_test = Resize((224, 224), interpolation=InterpolationMode.BILINEAR)(test_features)

        X_test, y_test_trues = X_test.to(device), y_test_trues.to(device)

        y_test_preds = model(X_test)

        _, y_test_preds_tags = torch.max(y_test_preds, dim=1)
        y_test_pred_list.append(y_test_preds_tags.cpu().numpy())

y_test_pred_list = [a.squeeze().tolist() for a in y_test_pred_list]
idx_to_class = {0: 'None', 1: 'C', 2: 'M', 3: 'T'}
y_test_pred_list= [idx_to_class[item] for sublist in y_test_pred_list for item in sublist]

In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_test, y_test_pred_list))
sns.heatmap(confusion_matrix_df, annot=True)

In [ ]:
print(classification_report(y_test, y_test_pred_list))